In [128]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import numpy as np

In [129]:
titles = []; trans = []; sales = [] ; costs = [] ; prices = [] ; mileages = []
# 매 페이지마다 공통적으로 변함없이 들어가 있는 정보들, 다시 말하면 css셀렉터를 문제없이 사용할 수 있다.

""" authors = []; editors = []; dates = []; pagenums = []; ISBNs = []; codes = [] """
# 상세 정보들이 페이지마다 '번역'이나 '구판정보' 처럼 있는 게 있고 없는 게 있기 때문에 동일한 css셀렉터를 쓸 수 없다. 매 페이지마다 바뀌기 때문이다.
# 그래서 딕셔너리로 뽑아서 한 책 마다 데이터프레임을 만들고 나중에 최종적으로 pd.concat([dfs])을 하면
# 최고 장점: 있는 열엔 알아서 아래로 붙고, 없는 열에는 알아서 NaN값으로 붙기 때문에 상세정보 경우를 나눌 필요가 없다.

finalDF = []
# 한 책마다 상세 정보들의 DataFrame을 담아놓는 리스트

for page in range(1,16+1):
    url = "https://www.hanbit.co.kr/academy/books/new_book_list.html"
    pageurl = url + '?page=' + str(page)
    
    res = requests.get(pageurl, headers = {'User-Agent':'Mozilla/5.0'})
    raw = res.text
    soup = BeautifulSoup(raw,'html.parser')
    # 1, 2, 3...대문 페이지의 soup

    frame = soup.select_one('.sub_book_list_area')
    cards = frame.select('.view_box')
    # 매 책의 프레임과 그 안의 카드

    bookurl = cards[0].select_one('.pop_quick_bg').attrs['onclick'][10:-1]
    # 카드의 onclick으로 매 책마다 주소 가져오기

    for card in cards:
        # 카드마다
        bookhref = card.select_one('.pop_quick_bg').attrs['onclick'][10:-1]
        suburl = 'https://www.hanbit.co.kr' + bookhref
        # 책의 주소 가져와서

        subres = requests.get(suburl, headers = {'User-Agent':'Mozilla/5.0'})
        subraw = subres.text
        subsoup = BeautifulSoup(subraw,'html.parser')
        # 각 책 페이지의 soup 
        
        # 가격 정보 박스
        pricebox = subsoup.select('span.pbr')

        cost = pricebox[0].text
        price = pricebox[1].text
        mileage = pricebox[2].text

        # 책 정보 박스
        infobox = subsoup.select_one('.store_product_info_box')

        title = infobox.select_one('h3').text
        tran = infobox.select('.info_ico p')[1].text
        sale = infobox.select('.info_ico p')[2].text

        # 상세 정보 박스
        infos = infobox.select('.info_list li')

        rowdic = {}
        # 각 책 페이지의 상세 정보 마다 돌아가며 딕셔너리를 만들어야 하기 때문에 반복문 또 한번
        for info in infos:
            # 상세 정보가 Dictionary로 만들기 딱 좋게
            # '번역 : 김성훈' 으로 되어있었음
            # 키, 밸류값으로 나눠주기

            info_key = info.text.split(':')[0]
            info_value = info.text.split(':')[1]

            # 그리고 딕셔너리로 만들어 주면
            rowdic[info_key] = [info_value]

        # 반복문 끝나고 한 페이지의 상세정보를 담는 딕셔너리가 완성되었음
        rowdf = pd.DataFrame(rowdic)
        # 딕셔너리로 데이터프레임을 만들면 상세정보를 열로 가지고, 행이 한 줄인 데이터프레임이 만들어짐
        # 한 페이지의 데이터 프레임을 최종 모든 데이터 프레임을 담는 리스트에 담아줌
        finalDF.append(rowdf)

        # 얘들은 css셀렉터로 가져올 수 있는 애들
        titles.append(title) ; trans.append(tran) ; sales.append(sale) ; costs.append(cost) ; prices.append(price) ; mileages.append(mileage) 

In [130]:
""" 책 프레임과 카드의 css를 구하는 과정 """
# frame = soup.select_one('.sub_book_list_area')
# cards = frame.select('.view_box')

In [131]:
""" 책 카드 onclick의 주소를 구하는 과정 """
# bookurl = cards[0].select_one('.pop_quick_bg').attrs['onclick'][10:-1]

In [132]:
""" 한 페이지마다 모든 onclick 주소를 구하는 과정 """
# for card in cards:
#     bookhref = card.select_one('.pop_quick_bg').attrs['onclick'][10:-1]
#     suburl = 'https://www.hanbit.co.kr' + bookhref
#     print(suburl)

In [135]:
""" 각 페이지마다 상세정보가 달라 css셀렉터를 쓰지 못한다는 걸 깨달은 과정 """
# titles = []; trans = []; sales = []; authors = []; editors = []; dates # = []; pagenums = []; ISBNs = []; codes = []
# 
# for card in cards:
#     bookhref = card.select_one('.pop_quick_bg').attrs['onclick'][10:-1]
#     suburl = 'https://www.hanbit.co.kr' + bookhref
#     subres = requests.get(suburl, headers = {'User-Agent':'Mozilla/5.0'})
#     subraw = subres.text
#     subsoup = BeautifulSoup(subraw,'html.parser')
# 
#     # 각 페이지
#     infobox = subsoup.select_one('.store_product_info_box')
#     
#     title = infobox.select_one('h3').text
#     tran = infobox.select('.info_ico p')[1].text
#     sale = infobox.select('.info_ico p')[2].text
# 
#     if trans == '번역서':
#         author = infobox.select('.info_list li')[0].text[5:-1]
#         editor = infobox.select('.info_list li')[1].text[5:-1]
#         date = infobox.select('.info_list li')[2].text[5:]
#         pagenum = infobox.select('.info_list li')[3].text[6:-2]
#         ISBN = infobox.select('.info_list li')[4].text[7:]
#         code = infobox.select('.info_list li')[5].text[-4:]
#     else:
#         author = infobox.select('.info_list li')[0].text[5:-1]
#         editor = np.nan
#         date = infobox.select('.info_list li')[1].text[5:]
#         pagenum = infobox.select('.info_list li')[2].text[6:-2]
#         ISBN = infobox.select('.info_list li')[3].text[7:]
#         code = infobox.select('.info_list li')[4].text[-4:]
#     
#     titles.append(title) ; trans.append(tran) ; sales.append(sale) ; authors.append(author) ; editors.append(editor) ; dates.append(date) ; pagenums.append(pagenum) ; ISBNs.append(ISBN) ; codes.append(code) 


In [145]:
""" 가격 박스 셀렉터 찾는 과정 """
# pricebox = subsoup.select('span.pbr')
# pricebox[2].text

' 가격 박스 셀렉터 찾는 과정 '

In [134]:
""" 각 페이지 상세정보가 딕셔너리처럼 생겼던 걸 알아낸 과정 """
# for info in infobox.select('.info_list li'):
#     print(info.text.split(':'))

In [144]:
""" 첫 페이지만 크롤링해서 성공했을 때, 모든 페이지를 돌아갈 때는 아래 코드 중 첫 for문 이하부터 붙이면 된다.  """
# titles = []; trans = []; sales = [] ; costs = [] ; prices = [] ; mileages = []
# # authors = []; editors = []; dates = []; pagenums = []; ISBNs = []; codes = []
# finalDF = []
# 
# for i, card in enumerate(cards):
#     bookhref = card.select_one('.pop_quick_bg').attrs['onclick'][10:-1]
#     suburl = 'https://www.hanbit.co.kr' + bookhref
#     subres = requests.get(suburl, headers = {'User-Agent':'Mozilla/5.0'})
#     subraw = subres.text
#     subsoup = BeautifulSoup(subraw,'html.parser')
# 
#     # 각 페이지
#     # 가격
#     pricebox = subsoup.select('span.pbr')
#     
#     cost = pricebox[0].text
#     price = pricebox[1].text
#     mileage = pricebox[2].text
# 
#     # 정보
#     infobox = subsoup.select_one('.store_product_info_box')
#     
#     title = infobox.select_one('h3').text
#     tran = infobox.select('.info_ico p')[1].text
#     sale = infobox.select('.info_ico p')[2].text
# 
#     infos = infobox.select('.info_list li')
# 
#     # 그 딕셔너리에 넣는거!
# 
#     rowdic = {}
#     for info in infos:
#         info_key = info.text.split(':')[0]
#         info_value = info.text.split(':')[1]
#         
#         # print(info_key,info_value)
#         rowdic[info_key] = [info_value]
#     
#     #print(rowdic)
#     
#     rowdf = pd.DataFrame(rowdic)
#     finalDF.append(rowdf)
#     
#     
#     titles.append(title) ; trans.append(tran) ; sales.append(sale) ; costs.append(cost) ; prices.append(price) ; mileages.append(mileage) 


' 첫 페이지만 크롤링해서 성공했을 때, 모든 페이지를 돌아갈 때는 아래 코드 중 첫 for문 이하부터 붙이면 된다.  '

In [138]:
""" DataFrame으로 얻어낸 상세정보에 css셀렉터+리스트로 얻어낸 상세정보를 열로 추가하는 과정  """
finalDF = pd.concat(finalDF)
finalDF['제목'] = titles
finalDF['번역서'] = trans
finalDF['판매여부'] = sales
finalDF['원가'] = costs
finalDF['판매가'] = prices
finalDF['마일리지'] = mileages
finalDF.head()

,저자,번역,출간,페이지,ISBN,물류코드,구판정보,개정판정보,제목,번역서,판매여부,원가,판매가,마일리지
0,"Behrouz A. Forouzan , Richard F. Gilberg",윤인성,2020-11-10,756 쪽,9791156645061,4506,NaN,NaN,포르잔 C++ 에센셜,번역서,판매중,"32,000원","32,000원(0% off)",960점 (3%)
0,김지연,NaN,2020-09-28,536 쪽,9791156645054,4505,"이 도서는 <IT CookBook, 데이터베이스 기초와 실습",NaN,"IT CookBook, 액세스 2019로 배우는 데이터베이스 기초와 실습",집필서,판매중,"28,000원","28,000원(0% off)",840점 (3%)
0,"Behrouz A. Forouzan , Richard F. Gilberg",윤인성,2020-08-31,1192 쪽,9791156645030,4503,NaN,NaN,포르잔 C++ 바이블,번역서,판매중,"45,000원","45,000원(0% off)","1,350점 (3%)"
0,Gilbert Strang,부산대학교 산업수학센터(FFMIMC),2020-08-05,560 쪽,9791156644972,4497,NaN,NaN,딥러닝을 위한 선형대수학,번역서,판매중,"38,000원","38,000원(0% off)","1,140점 (3%)"
0,"박우창 , 남송휘 , 이현룡",NaN,2020-07-30,600 쪽,9791156645023,4502,"이 도서는 <IT CookBook, 오라클로 배우는 데이터베이스 개론과 실습>의 개...",NaN,"IT CookBook, 오라클로 배우는 데이터베이스 개론과 실습(2판)",집필서,판매중,"29,000원","29,000원(0% off)",870점 (3%)


In [139]:
""" 데이터 프레임의 열 순서 바꾸가 """
finalDF.columns

Index(['저자 ', '번역 ', '출간 ', '페이지 ', 'ISBN ', '물류코드 ', '구판정보 ', '개정판정보 ', '제목',
       '번역서', '판매여부', '원가', '판매가', '마일리지'],
      dtype='object')

In [140]:
newDf = finalDF.reindex(columns=[
    '제목',
    '판매여부',
    '저자 ',
    '원가',
    '판매가',
    '마일리지',
    '번역서',
    '번역 ',
    '출간 ',
    '페이지 ',
    '구판정보 ',
    'ISBN ',
    '물류코드 '
])
newDf.head()

,제목,판매여부,저자,원가,판매가,마일리지,번역서,번역,출간,페이지,구판정보,ISBN,물류코드
0,포르잔 C++ 에센셜,판매중,"Behrouz A. Forouzan , Richard F. Gilberg","32,000원","32,000원(0% off)",960점 (3%),번역서,윤인성,2020-11-10,756 쪽,NaN,9791156645061,4506
0,"IT CookBook, 액세스 2019로 배우는 데이터베이스 기초와 실습",판매중,김지연,"28,000원","28,000원(0% off)",840점 (3%),집필서,NaN,2020-09-28,536 쪽,"이 도서는 <IT CookBook, 데이터베이스 기초와 실습",9791156645054,4505
0,포르잔 C++ 바이블,판매중,"Behrouz A. Forouzan , Richard F. Gilberg","45,000원","45,000원(0% off)","1,350점 (3%)",번역서,윤인성,2020-08-31,1192 쪽,NaN,9791156645030,4503
0,딥러닝을 위한 선형대수학,판매중,Gilbert Strang,"38,000원","38,000원(0% off)","1,140점 (3%)",번역서,부산대학교 산업수학센터(FFMIMC),2020-08-05,560 쪽,NaN,9791156644972,4497
0,"IT CookBook, 오라클로 배우는 데이터베이스 개론과 실습(2판)",판매중,"박우창 , 남송휘 , 이현룡","29,000원","29,000원(0% off)",870점 (3%),집필서,NaN,2020-07-30,600 쪽,"이 도서는 <IT CookBook, 오라클로 배우는 데이터베이스 개론과 실습>의 개...",9791156645023,4502


In [141]:
""" 데이터프레임 열 이름 바꾸기 """
newDf.rename(columns={
    '제목' : '제목',
    '판매여부' : '판매여부',
    '저자 ' : '저자',
    '원가':'원가',
    '판매가':'판매가',
    '마일리지':'마일리지',
    '번역서' : '번역여부',
    '번역 ' : '엮은이',
    '출간 ' : '출간일',
    '페이지 ' : '페이지',
    '구판정보 ' : '구판여부',
    'ISBN ' : 'ISBN',
    '물류코드 ':'물류코드'
},inplace=True)

In [142]:
newDf

,제목,판매여부,저자,원가,판매가,마일리지,번역여부,엮은이,출간일,페이지,구판여부,ISBN,물류코드
0,포르잔 C++ 에센셜,판매중,"Behrouz A. Forouzan , Richard F. Gilberg","32,000원","32,000원(0% off)",960점 (3%),번역서,윤인성,2020-11-10,756 쪽,NaN,9791156645061,4506
0,"IT CookBook, 액세스 2019로 배우는 데이터베이스 기초와 실습",판매중,김지연,"28,000원","28,000원(0% off)",840점 (3%),집필서,NaN,2020-09-28,536 쪽,"이 도서는 <IT CookBook, 데이터베이스 기초와 실습",9791156645054,4505
0,포르잔 C++ 바이블,판매중,"Behrouz A. Forouzan , Richard F. Gilberg","45,000원","45,000원(0% off)","1,350점 (3%)",번역서,윤인성,2020-08-31,1192 쪽,NaN,9791156645030,4503
0,딥러닝을 위한 선형대수학,판매중,Gilbert Strang,"38,000원","38,000원(0% off)","1,140점 (3%)",번역서,부산대학교 산업수학센터(FFMIMC),2020-08-05,560 쪽,NaN,9791156644972,4497
0,"IT CookBook, 오라클로 배우는 데이터베이스 개론과 실습(2판)",판매중,"박우창 , 남송휘 , 이현룡","29,000원","29,000원(0% off)",870점 (3%),집필서,NaN,2020-07-30,600 쪽,"이 도서는 <IT CookBook, 오라클로 배우는 데이터베이스 개론과 실습>의 개...",9791156645023,4502
...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,문제해결력을 키우는 이공계 글쓰기 : 발표와 토론까지,판매중,한성우,"18,000원","18,000원(0% off)",540점 (3%),집필서,NaN,2013-02-28,396 쪽,NaN,9788998756024,4002
0,"진로개발과 직업세계 : 대학생과 청년층을 위한 진로, 취업지침서",판매중,"서일수 , 김도균 , 김지혜 , 이정배","20,000원","20,000원(0% off)",600점 (3%),집필서,NaN,2013-02-28,392 쪽,NaN,9788998756048,4004
0,Essential Calculus,판매중,James Stewart,"36,000원","36,000원(0% off)","1,080점 (3%)",원서,NaN,2012-07-08,960 쪽,NaN,9781133492566,7999
0,"IT CookBook, 디지털 통신 시스템 : Digital Communicatio...",판매중,"정호영 , 임승각","25,000원","25,000원(0% off)",750점 (3%),집필서,NaN,2011-01-31,576 쪽,NaN,9788979147988,1798


In [143]:
newDf.to_csv('book.csv',sep=',',encoding='utf-8')